# Mixed-Integer Linear Programming

Importing gurobi MIP solver and other required libraries

In [15]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import os

from utils.utils import load_data, plot_device

## Variant 1 - no rotation

In [24]:
def mip_solve_1(w, n, widths, heights):
    # Create a new model
    m = gp.Model('vlsi_v1')
    
    # Create variables
    coordinates_x = [m.addVar(vtype='I', name=f'x_{i}') for i in range(n)]
    coordinates_y = [m.addVar(vtype='I', name=f'y_{i}') for i in range(n)]
    max_heights = [m.addVar(vtype='I', name=f'max_y_{i}') for i in range(n)]
    max_height = m.addVar(vtype='I', name='max_height')
    
    # Add constraints
    # Maximum heights
    m.addConstrs((max_heights[i] == coordinates_y[i] + heights[i] for i in range(n)))
    # Maximum height
    m.addConstr(max_height == gp.max_(max_heights))
    
    # Total width constraint
    for i in range(n):
        m.addConstr(coordinates_x[i] + widths[i] <= w)
    
    # No overlap constraints
    for i in range(n):
        for j in range(i + 1, n):
            j_is_left = m.addVar(vtype='B', name=f'j_is_left_{i}_{j}')
            m.addGenConstrIndicator(j_is_left, True, coordinates_x[i] >= coordinates_x[j] + widths[j])

            j_is_right = m.addVar(vtype='B', name=f'j_is_right_{i}_{j}')
            m.addGenConstrIndicator(j_is_right, True, coordinates_x[i] + widths[i] <= coordinates_x[j])

            not_overlaps_horizontal = m.addVar(vtype='B', name=f'not_overlaps_horizontal_{i}_{j}')
            m.addGenConstrIndicator(not_overlaps_horizontal, True, j_is_left + j_is_right >= 1)

            j_is_down = m.addVar(vtype='B', name=f'j_is_down_{i}_{j}')
            m.addGenConstrIndicator(j_is_down, True, coordinates_y[i] >= coordinates_y[j] + heights[j])

            j_is_up = m.addVar(vtype='B', name=f'j_is_up_{i}_{j}')
            m.addGenConstrIndicator(j_is_up, True, coordinates_y[i] + heights[i] <= coordinates_y[j])

            not_overlaps_vertical = m.addVar(vtype='B', name=f'not_overlaps_vertical_{i}_{j}')
            m.addGenConstrIndicator(not_overlaps_vertical, True, j_is_down + j_is_up >= 1)

            m.addConstr(not_overlaps_horizontal + not_overlaps_vertical >= 1)
    
    # Set objective function
    m.setObjective(max_height, gp.GRB.MINIMIZE)
    
    # Solve it!
    m.Params.output_flag = 0
    m.Params.time_limit = 300.0
    m.optimize()
    
    return m.getAttr('runtime'), [v.X for v in coordinates_x], [v.X for v in coordinates_y], max_height.X


In [25]:
def execute(instance, save_sol=True, write_stats=True, display_image=True, save_image=True):
    path = os.path.abspath('')
    
    w, n, widths, heights = load_data(instance)
    
    runtime, c_x, c_y, h = mip_solve_1(w, n, widths, heights)
    print(f'Height of the silicon plate: {h}')
    
    #if save_sol:
        #write_sol(path + f'/out/{instance}.txt', w, n, widths, heights, c_x, c_y, h)
    
    if display_image:
        plot_device(c_x, c_y, widths, heights, max(w, h), max(w, h))
    
    if save_image:
        plot_device(c_x, c_y, widths, heights, max(w, h), max(w, h), path + f'/img/{instance}.png')
    
    return runtime
        

In [28]:
r = execute(1, display_image=False)

print(f'Execution time: {r}')

Height of the silicon plate: 8.0
Execution time: 0.012998580932617188


<Figure size 640x480 with 0 Axes>